# Post Training Quantization a YoloV8-nano Object Detection Model

[Run this tutorial in Google Colab](https://colab.research.google.com/github/sony/model_optimization/blob/main/tutorials/notebooks/example_keras_yolov8n.ipynb)

## Overview


In this tutorial, we'll demonstrate the post-training quantization using MCT for a pre-trained object detection model in Keras. Specifically, we'll integrate post-processing, including the non-maximum suppression (NMS) layer, into the model. This integration aligns with the imx500 target platform capabilities.

In this example we will use an existing pre-trained YoloV8-nano model taken from [Ultralytics](https://github.com/ultralytics/ultralytics). We will convert the model to a Tensorflow model that includes box decoding and NMS layer. Further, we will quantize the model using MCT post training quantization and evaluate the performance of the floating point model and the quantized model on COCO dataset.


## Summary

In this tutorial we will cover:

1. Post-Training Quantization using MCT of Keras object detection model including the post-processing.
2. Data preparation - loading and preprocessing validation and representative datasets from COCO.
3. Accuracy evaluation of the floating-point and the quantized models.

## Setup
Install the relevant packages.

In [ ]:
!pip install -q torch
!pip install -q tensorflow
!pip install -q pycocotools
!pip install -q ultralytics
!pip install -q model-compression-toolkit

 Clone a copy of the [MCT](https://github.com/sony/model_optimization) (Model Compression Toolkit) into your current directory. This step ensures that you have access to the tutorials [resources](https://github.com/sony/model_optimization/tree/main/tutorials/resources) folder which contains all the necessary utility functions for this tutorial.

In [ ]:
!git clone https://github.com/sony/model_optimization.git local_mct
import sys
sys.path.insert(0,"/content/local_mct")

Finally, load COCO evaluation set

In [ ]:
!wget -nc http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip -q -o annotations_trainval2017.zip -d /content/coco
!echo Done loading annotations
!wget -nc http://images.cocodataset.org/zips/val2017.zip
!unzip -q -o val2017.zip -d /content/coco
!echo Done loading val2017 images

## Floating Point Model

### Load the pre-trained weights of Yolo8-nano
We begin by loading the pre-trained weights of `YOLOv8n` from [Ultralytics](https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8n.pt) using `torch.load`, as the original model is in PyTorch format. Please make sure the pre-trained weights are located in the `content` directory or specify the correct path.

In [ ]:
!wget -nc https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8n.pt
import torch

PRETRAINED_WEIGHTS_FILE = '/content/yolov8n.pt'
pretrained_weights = torch.load(PRETRAINED_WEIGHTS_FILE)['model'].state_dict()

### Generate Yolo8-nano Keras model
In the following steps, we proceed to create a Keras model based on [Ultralytics](https://github.com/ultralytics/ultralytics). This implementation includes a slightly modified version of yolov8
detection-head (mainly the box decoding part) that was optimized for model quantization. We then initialize the model weights to match those of the original model.

To this base model, we integrate the post-processing components, which include tensorflow [tf.image.combined_non_max_suppression](https://www.tensorflow.org/api_docs/python/tf/image/combined_non_max_suppression) layer.
For further insights into the model's implementation details, please refer to the tutorials ["resources" folder](https://github.com/sony/model_optimization/tree/main/tutorials/resources).   

In [ ]:
import tensorflow as tf
from keras.models import Model
from tutorials.resources.utils.torch2keras_weights_translation import load_state_dict
from tutorials.resources.yolov8.yolov8_keras import yolov8_keras

# Parameter of Yolov8n
INPUT_RESOLUTION = 640

# Generate Yolov8n model 
model = yolov8_keras('/content/local_mct/tutorials/resources/yolov8/yolov8n.yaml', INPUT_RESOLUTION)

# Set the pre-trained weights
load_state_dict(model, state_dict_torch=pretrained_weights)

# Add Tensorflow NMS layer
boxes, scores = model.output
outputs = tf.image.combined_non_max_suppression(
    boxes,
    scores,
    max_output_size_per_class=300,
    max_total_size=300,
    iou_threshold=0.7,
    score_threshold=0.001,
    pad_per_class=False,
    clip_boxes=False
    )

model = Model(model.input, outputs, name='yolov8n')

print('Model is ready for evaluation')

#### Evaluate the floating point model
Next, we evaluate the floating point model by using `cocoeval` library alongside additional dataset utilities. We can verify the mAP accuracy aligns with that of the original model. 
Note that we set the "batch_size" to 5 and the preprocessing according to [Ultralytics](https://github.com/ultralytics/ultralytics).
Please ensure that the dataset path has been set correctly before running this code cell.

In [ ]:
from tutorials.resources.utils.coco_evaluation import coco_dataset_generator, CocoEval
from tutorials.resources.yolov8.yolov8_keras import yolov8_preprocess

EVAL_DATASET_FOLDER = '/content/coco/val2017'
EVAL_DATASET_ANNOTATION_FILE = '/content/coco/annotations/instances_val2017.json'
BATCH_SIZE = 5

# Load COCO evaluation set
val_dataset = coco_dataset_generator(dataset_folder=EVAL_DATASET_FOLDER,
                                     annotation_file=EVAL_DATASET_ANNOTATION_FILE,
                                     preprocess=yolov8_preprocess,
                                     batch_size=BATCH_SIZE)

# Define resizing information to map between the model's output and the original image dimensions
output_resize = {'shape': (INPUT_RESOLUTION, INPUT_RESOLUTION), 'aspect_ratio_preservation': True}

# Initialize the evaluation metric object
coco_metric = CocoEval(EVAL_DATASET_ANNOTATION_FILE, output_resize) 

# Iterate and the evaluation set
for batch_idx, (images, targets) in enumerate(val_dataset):
    
    # Run inference on the batch
    outputs = model(images)

    # Add the model outputs to metric object (a dictionary of outputs after postprocess: boxes, scores & classes)
    coco_metric.add_batch_detections(outputs, targets)
    if (batch_idx + 1) % 100 == 0:
        print(f'processed {(batch_idx + 1) * BATCH_SIZE} images')

# Print float model mAP results
print("Float model mAP: {:.4f}".format(coco_metric.result()[0]))

## Quantize Model

### Post training quantization using Model Compression Toolkit 

Now, we're all set to use MCT's post-training quantization. To begin, we'll define a representative dataset and proceed with the model quantization. Please note that, for demonstration purposes, we'll use the evaluation dataset as our representative dataset. We'll calibrate the model using 100 representative images, divided into 20 iterations of 'batch_size' images each. In addition, in order to further compress the model memory footprint, we will use mixed precision quantization technique instead of fixed 8 bits quantization. 

In [ ]:
import model_compression_toolkit as mct
from typing import Iterator, Tuple, List

REPRESENTATIVE_DATASET_FOLDER = '/content/coco/val2017/'
REPRESENTATIVE_DATASET_ANNOTATION_FILE = '/content/coco/annotations/instances_val2017.json'
n_iters = 20

# Load representative dataset
representative_dataset = coco_dataset_generator(dataset_folder=REPRESENTATIVE_DATASET_FOLDER,
                                                annotation_file=REPRESENTATIVE_DATASET_ANNOTATION_FILE,
                                                preprocess=yolov8_preprocess,
                                                batch_size=BATCH_SIZE)

# Define representative dataset generator
def get_representative_dataset(n_iter: int, dataset_loader: Iterator[Tuple]):
    """
    This function creates a representative dataset generator.
    Args:
        n_iter: number of iterations for MCT to calibrate on
    Returns:
        A representative dataset generator
    """       
    def representative_dataset() -> Iterator[List]:
        """
        Creates a representative dataset generator from a PyTorch data loader, The generator yields numpy
        arrays of batches of shape: [Batch, H, W ,C].
        Returns:
            A representative dataset generator
        """
        ds_iter = iter(dataset_loader)
        for _ in range(n_iter):
            yield [next(ds_iter)[0]]

    return representative_dataset

# Get representative dataset generator
representative_dataset_gen = get_representative_dataset(n_iters, representative_dataset)

# Set IMX500-v1 TPC
tpc = mct.get_target_platform_capabilities("tensorflow", 'imx500', target_platform_version='v1')

# Specify the necessary configuration for mixed precision quantization. To keep the demonstration brief, we'll use a small set of images and omit the hessian metric for mixed precision calculations.   
mp_config = mct.core.MixedPrecisionQuantizationConfigV2(num_of_images=5, use_hessian_based_scores=False)
config = mct.core.CoreConfig(mixed_precision_config=mp_config,
                             quantization_config=mct.QuantizationConfig(shift_negative_activation_correction=True))

# Define memory KPI for mixed precision weights quantization (75% of 'standard' 8bits quantization)
kpi_data = mct.core.keras_kpi_data_experimental(model,
                                                representative_dataset_gen,
                                                config,
                                                target_platform_capabilities=tpc)
kpi = mct.core.KPI(kpi_data.weights_memory * 0.75)

# Preform post training quantization
quant_model, _ = mct.ptq.keras_post_training_quantization_experimental(model,
                                                                       representative_dataset_gen,
                                                                       target_kpi=kpi,
                                                                       core_config=config,
                                                                       target_platform_capabilities=tpc)
print('Quantized model is ready')

### Evaluate quantized model
Lastly, we can evaluate the performance of the quantized model. There is a slight decrease in performance that can be further mitigated by either expanding the representative dataset or employing MCT's advanced quantization methods, such as GPTQ (Gradient-Based/Enhanced Post Training Quantization).

In [ ]:
# Re-load COCO evaluation set
val_dataset = coco_dataset_generator(dataset_folder=EVAL_DATASET_FOLDER,
                                     annotation_file=EVAL_DATASET_ANNOTATION_FILE,
                                     preprocess=yolov8_preprocess,
                                     batch_size=BATCH_SIZE)

# Initialize the evaluation metric object
coco_metric = CocoEval(EVAL_DATASET_ANNOTATION_FILE, output_resize) 

# Iterate and the evaluation set
for batch_idx, (images, targets) in enumerate(val_dataset):
    # Run inference on the batch
    outputs = quant_model(images)

    # Add the model outputs to metric object (a dictionary of outputs after postprocess: boxes, scores & classes)
    coco_metric.add_batch_detections(outputs, targets)
    if (batch_idx + 1) % 100 == 0:
        print(f'processed {(batch_idx + 1) * BATCH_SIZE} images')

# Print quantized model mAP results
print("Quantized model mAP: {:.4f}".format(coco_metric.result()[0]))